# Numerical Simulation of Ballistic Performance of Honeycomb Sandwich Panels

### Objective:
Setup a parametric model in PyMAPDL for a honeycomb sandwich panel. Use it for running a Ballistic impact simulation

The learning objectives of this demo are:
* Setup and solve a parametric model using PyMAPDL
* Interactive plotting of CAD, mesh, and results in Pythonic interface.

### Model Details

#### Model parameters:
* Cell size
* Cell wall thickness
* Node length
* Facesheet thickness

## Step 1 - define all parameters

In [1]:
## Main parameters
cell_size = 0.006
cell_wall_thickness = 0.002
node_length = 0.004
facesheet_thickness = 0.005

In [2]:
# Fixed constants
structure_length = 0.1
structure_breadth = 0.1

In [3]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Additional calculation 
L = cell_size/ np.sqrt(3)
offset_x = cell_size + cell_wall_thickness
offset_y = 1.5*L + cell_wall_thickness*(np.sin(np.pi/3))
Nx = int(structure_length / offset_x) + 1
Ny = int(structure_breadth / offset_y) + 1
print("L = ", L)
print("offset_x = ", offset_x)
print("offset_y = ", offset_y)
print("Nx = ", Nx)
print("Ny = ", Ny)

L =  0.0034641016151377548
offset_x =  0.008
offset_y =  0.0069282032302755096
Nx =  13
Ny =  15


# Step 2 - launch MAPDL and create geometry

In [4]:
from ansys.mapdl.core import launch_mapdl

# start mapdl
mapdl = launch_mapdl()
print(mapdl)

PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...
Product:             Ansys Mechanical Enterprise
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [5]:
## reset mapdl
mapdl.clear()

## enter the preprocessor
mapdl.prep7()

mapdl.units('SI')      # SI unit system

## Create geometry
mapdl.block(0, structure_length, 0, structure_breadth, 0, node_length)

plate = mapdl.cm("PLATE", "VOLU")

mapdl.clocal( 11, 0, structure_length/2, structure_breadth/2, 0)                                         ## Making local coordinate system
mapdl.wpcsys('', 11,)                                                                                    ## Shift working plane

initial_distance = node_length + 0.02                                              ## 20 mm from center of front facesheet of sandwich panel

# 0.44 Remington Magnum Bullet
mapdl.cone(0.00545, 0.005486, initial_distance , initial_distance + 0.00825)
mapdl.cone(0.00569, 0.005805, initial_distance + 0.00825 , initial_distance + 0.04089 )
mapdl.vadd(1,2)
bullet = mapdl.cm("BULLET","VOLU")

mapdl.csdele(11)                                                                   ## Delete local coordinate system
# mapdl.input('bullet9.anf')

mapdl.vplot('all')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 3 - define material properties, mesh attributes and generate mesh

In [6]:
## Define material properties
mapdl.mp("EX", 1, 2e5)  # Youngs modulus
mapdl.mp("PRXY", 1, 0.3367)  # Poissons ratio

mapdl.tb("EVISC",1)

## Define element attributes
mapdl.et(1, "SOLID185") # 3D 8-Node Layered Solid

## Define mesh controls

mapdl.lesize("ALL", 0.001, layer1=1)

# Mesh facesheets
mapdl.mshape(0, "3D")    # mesh the volume with 3D Hexahedral elements
mapdl.mshkey(1)          # mapped mesh
mapdl.vmesh('PLATE')

# # Mesh bullet
mapdl.mshape(1, "3D")    # mesh the volume with 3D Tetrahedral elements
mapdl.mshkey(0)          # free mesh
mapdl.vmesh('BULLET')

## Plot the mesh
mapdl.eplot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Step 4 - apply loads and boundary conditions

In [7]:
# from ansys.dyna.core.pre.dynasolution import DynaSolution

In [8]:
### Make the lateral sides of the panel as fixed support i.e. set displacement of nodes to zero

# Select all nodes at the x = 0 end of the sandwich panel
mapdl.nsel("S", "LOC", "X", 0)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the x = structure_length end of the sandwich panel
mapdl.nsel("S", "LOC", "X", structure_length)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the y = 0 end of the sandwich panel
mapdl.nsel("S", "LOC", "Y", 0)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# Select all nodes at the y = structure_breadth end of the sandwich panel
mapdl.nsel("S", "LOC", "Y", structure_breadth)
# Remove all degrees of freedom for all nodes in the selection
mapdl.d("all", "all")

# # plot the boundary conditions
# mapdl.nplot(plot_bc=True)

# mapdl.edpart('ADD', partid='1', cname='BULLET')
# mapdl.edpvel(option='VELO', pid='1', vz='400')
# mapdl.edpvel('VELO', 'BULLET', vz='-400')

SPECIFIED CONSTRAINT UX   FOR SELECTED NODES            1 TO       58787 BY           1
 REAL=  0.00000000       IMAG=  0.00000000    
 ADDITIONAL DOFS=  UY    UZ

## Step 5 - Simulation and Results

In [10]:
# from ansys.dyna.core import pre
# dyna_mech = pre.DynaDEM

In [ ]:
from ansys.mapdl.core import Report
print(Report())

In [11]:
# dyna_mech.generate_keywords()


AttributeError: type object 'DynaDEM' has no attribute 'generate_keywords'

In [12]:
# mapdl.edpvel('VELO', 'BULLET', vz='-400')

MapdlRuntimeError: 

Error in instance GRPC_127.0.0.1:50052

*** ERROR ***                           CP =       1.828   TIME= 14:59:42
 The capability LsDyna was either not ordered with this Ansys            
 installation or the appropriate product was not selected for this       
 session.                                                                
  Contact your Ansys support person for more information.

In [ ]:
# mapdl.run("/SOLU")
# mapdl.antype("TRANS")
# mapdl.run("/STATUS,SOLU")

S O L U T I O N   O P T I O N S

   PROBLEM DIMENSIONALITY. . . . . . . . . . . . .3-D                  
   DEGREES OF FREEDOM. . . . . . UX   UY   UZ  
   ANALYSIS TYPE . . . . . . . . . . . . . . . . .TRANSIENT            
      SOLUTION METHOD. . . . . . . . . . . . . . .FULL                 
   GLOBALLY ASSEMBLED MATRIX . . . . . . . . . . .SYMMETRIC  

                      L O A D   S T E P   O P T I O N S

   LOAD STEP NUMBER. . . . . . . . . . . . . . . .     1
   TIME AT END OF THE LOAD STEP. . . . . . . . . .  1.0000    
   NUMBER OF SUBSTEPS. . . . . . . . . . . . . . .     1
   STEP CHANGE BOUNDARY CONDITIONS . . . . . . . .DEFAULT
   TRANSIENT (INERTIA) EFFECTS
   TRANSIENT INTEGRATION PARAMETERS
      ALPHA. . . . . . . . . . . . . . . . . . . . 0.25251    
      DELTA. . . . . . . . . . . . . . . . . . . . 0.50500    
   PRINT OUTPUT CONTROLS . . . . . . . . . . . . .NO PRINTOUT
   DATABASE OUTPUT CONTROLS. . . . . . . . . . . .ALL DATA WRITTEN
                          

In [ ]:
# mapdl.edpvel('VELO', 'BULLET', vz='-400')

MapdlRuntimeError: 

Error in instance GRPC_127.0.0.1:50052

*** ERROR ***                           CP =       2.062   TIME= 14:42:29
 The capability LsDyna was either not ordered with this Ansys            
 installation or the appropriate product was not selected for this       
 session.                                                                
  Contact your Ansys support person for more information.

In [13]:
mapdl.exit()